# Курс «Машинное обучение в бизнесе»

## Урок 8. Кейс 2. Типы моделей для задачи оттока

### Домашнее задание к уроку 8


In [1]:
import pandas as pd
import numpy as np
import os
import matplotlib.pyplot as plt#visualization
from PIL import  Image
%matplotlib inline
import seaborn as sns#visualization
import itertools
import warnings
warnings.filterwarnings("ignore")
import io
import plotly.offline as py#visualization
py.init_notebook_mode(connected=True)#visualization
import plotly.graph_objs as go#visualization
import plotly.tools as tls#visualization
import plotly.figure_factory as ff#visualization

from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import confusion_matrix,accuracy_score,classification_report
from sklearn.metrics import roc_auc_score,roc_curve,scorer
from sklearn.metrics import f1_score
import statsmodels.api as sm
from sklearn.metrics import precision_score,recall_score
from yellowbrick.classifier import DiscriminationThreshold

In [2]:
def split_data( data, split_date ):
    return data.loc[data['date'] <= split_date].copy(), data.loc[data['date'] > split_date].copy()

#### Задание 1

Скачать любой понравившийся вам датасет с UCI repository(можете даже свой какой-то взять) и сразу же разделите его на тренировочную и тестовую выборки. Тестовую выборку сохраните на диск и больше не трогайте.

Для лотереи "4 из 20" необходимо угадать величину приза.

Показатели:  
    **date** — дата тиража  
    **n_1_1** — первое число из 20 в первом блоке  
    **n_1_2** — второе число из 20 в первом блоке  
    **n_1_3** — третье число из 20 в первом блоке  
    **n_1_4** — четвертое число из 20 в первом блоке  
    **n_2_1** — первое число из 20 во втором блоке  
    **n_2_2** — второе число из 20 во втором блоке  
    **n_2_3** — третье число из 20 во втором блоке  
    **n_2_4** — четвертое число из 20 во втором блоке  
    **prize** — величина приза

Целевая переменная **prize**

In [3]:
lottery = pd.read_csv("lottery.csv", index_col=1, parse_dates=[0]).sort_index()
print(lottery.shape)
print ("Строк               : " , lottery.shape[0])
print ("Колонок             : " , lottery.shape[1])
print ("\nПоказатели          :\n" , lottery.columns.tolist())
print ("\nПустые значения     : ", lottery.isnull().sum().values.sum())
print ("\nУникальные значения :\n", lottery.nunique())
print ("\nСводка по набору    :")
lottery.info()
lottery.head(3)

(2173, 10)
Строк               :  2173
Колонок             :  10

Показатели          :
 ['date', 'n_1_1', 'n_1_2', 'n_1_3', 'n_1_4', 'n_2_1', 'n_2_2', 'n_2_3', 'n_2_4', 'prize']

Пустые значения     :  0

Уникальные значения :
 date     2173
n_1_1      16
n_1_2      17
n_1_3      17
n_1_4      17
n_2_1      16
n_2_2      17
n_2_3      17
n_2_4      17
prize    1992
dtype: int64

Сводка по набору    :
<class 'pandas.core.frame.DataFrame'>
Int64Index: 2173 entries, 1 to 2173
Data columns (total 10 columns):
 #   Column  Non-Null Count  Dtype         
---  ------  --------------  -----         
 0   date    2173 non-null   datetime64[ns]
 1   n_1_1   2173 non-null   int64         
 2   n_1_2   2173 non-null   int64         
 3   n_1_3   2173 non-null   int64         
 4   n_1_4   2173 non-null   int64         
 5   n_2_1   2173 non-null   int64         
 6   n_2_2   2173 non-null   int64         
 7   n_2_3   2173 non-null   int64         
 8   n_2_4   2173 non-null   int64         
 9  

,date,n_1_1,n_1_2,n_1_3,n_1_4,n_2_1,n_2_2,n_2_3,n_2_4,prize
circulation,,,,,,,,,,
1,2016-12-31 15:10:00,2,4,10,20,2,9,16,17,1 000 000 000
2,2017-03-01 22:00:00,6,12,17,20,3,9,13,16,300 000 000
3,2017-05-01 22:00:00,5,17,18,19,11,12,14,20,300 000 000


In [4]:
lottery.insert(0, 'id', lottery.index.values)

In [5]:
lottery['prize'] = lottery['prize'].str.replace(' ', '')

Выделим категориальные численные столбцы

In [6]:
#Separating catagorical and numerical columns
Id_col     = ['id']
target_col = ['prize']
cat_cols   = lottery.nunique()[lottery.nunique() < 6].keys().tolist()
cat_cols   = [x for x in cat_cols if x not in target_col]
num_cols   = [x for x in lottery.columns if x not in cat_cols + target_col + Id_col]
Id_col, target_col, cat_cols, num_cols

(['id'],
 ['prize'],
 [],
 ['date',
  'n_1_1',
  'n_1_2',
  'n_1_3',
  'n_1_4',
  'n_2_1',
  'n_2_2',
  'n_2_3',
  'n_2_4'])

Категориальных значений нет

Разобъем набор на тренировочный и тестовый наборы

In [7]:
train, test = split_data(lottery, '2020-01-01')

features = cat_cols+num_cols
target = target_col

X_test = test[features+['id']]
y_test = test[target]

X_train = train[features+['id']]
y_train = train[target]


X_test.to_csv("X_test.csv", index=None)
y_test.to_csv("y_test.csv", index=None)

X_train.to_csv("X_train.csv", index=None)
y_train.to_csv("y_train.csv", index=None)

In [8]:
X_train.head()

,date,n_1_1,n_1_2,n_1_3,n_1_4,n_2_1,n_2_2,n_2_3,n_2_4,id
circulation,,,,,,,,,,
1,2016-12-31 15:10:00,2,4,10,20,2,9,16,17,1
2,2017-03-01 22:00:00,6,12,17,20,3,9,13,16,2
3,2017-05-01 22:00:00,5,17,18,19,11,12,14,20,3
4,2017-08-01 08:20:00,5,12,17,19,3,6,7,8,4
5,2017-10-01 22:00:00,3,8,9,13,5,7,8,16,5


#### Задание 2

Провести EDA (exploratory data analysis: histograms, boxplots - visualization, etc).

In [9]:
# В работе

#### Задание 3

Сделать feature engineering. При этом постарайтесь «завернуть» ваш код для преобразования данных в классы или функции (пример Transformer как в ноутбуке).

In [10]:
# В работе

#### Задание 4

Обучить базовую модель (любую - на ваш выбор). При этом нужно использовать cross validation.

In [11]:
# В работе

#### Задание 5

Посчитайте разные метрики качества на cross-validation, отрисуйте графики (roc auc).

In [12]:
# В работе

#### Задание 6

Сделать подбор гиперпараметров с помощью той же самой cross-validation.

In [13]:
# В работе

#### Задание 7

Сохраните вашу модель на диск

In [14]:
# В работе

#### Задание 8

[в отдельном ноутбуке!] Загрузите ваш отложенный на шаге 1 датасет и сохраненные ранее файлы и сделайте прогноз моделью.

In [15]:
# В работе

#### Задание 9

Посчитайте метрики качества и сделайте выводы о том, насколько прогнозы (качество) на «новых» данных отличаются от того, что вы видели на cross-validation. Если будут очень серьезные отличия, то подумайте, где вы ошиблись и как это можно поправить.

In [16]:
# В работе